In [63]:
import numpy as np
from sklearn.datasets import fetch_mldata
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import cv2
from sklearn.metrics import classification_report, confusion_matrix
from collections import OrderedDict

In [76]:
mnist = fetch_mldata(dataname='mnist-original', data_name='D:/Python/HeThongThoiGianThuc/mldata')

X, y = mnist["data"], mnist["target"]
X = X / 255

digits = 2

x0 = X[np.where(y==0)[0]]
x1 = X[np.where(y==1)[0]]
y0 = np.zeros(x0.shape[0])
y1 = np.ones(x1.shape[0])
X = np.concatenate((x0, x1), axis = 0)
y = np.concatenate((y0, y1))
examples = y.shape[0]
y = y.reshape(1, examples)
print(y)

[[0. 0. 0. ... 1. 1. 1.]]


In [77]:
Y_new = np.eye(digits)[y.astype('int32')]

Y_new = Y_new.T.reshape(digits, examples)
print(Y_new)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]


In [78]:
m = 12000
m_test = X.shape[0] - m

X_train, X_test = X[:m].T, X[m:].T
Y_train, Y_test = Y_new[:,:m], Y_new[:,m:]

shuffle_index = np.random.permutation(m)
X_train, Y_train = X_train[:, shuffle_index], Y_train[:, shuffle_index]

In [79]:

def compute_multiclass_loss(Y, Y_hat):

    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1/m) * L_sum

    return L

In [80]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [81]:
n_x = X_train.shape[0]
n_h = 64
learning_rate = 1

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(digits, n_h)
b2 = np.zeros((digits, 1))

X = X_train
Y = Y_train

In [82]:
for i in range(2000):

    Z1 = np.matmul(W1,X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2,A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

    cost = compute_multiclass_loss(Y, A2)

    dZ2 = A2-Y
    dW2 = (1./m) * np.matmul(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./m) * np.matmul(dZ1, X.T)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)

    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

    if (i % 100 == 0):
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)

Z1 = np.matmul(W1, X_test) + b1
A1 = sigmoid(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

predictions = np.argmax(A2, axis=0)
print(predictions.reshape(-1,1))
labels = np.argmax(Y_test, axis=0)
print(labels)


print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))

Epoch 0 cost:  2.2890839116724875
Epoch 100 cost:  0.013824005948261319
Epoch 200 cost:  0.00908524394853885
Epoch 300 cost:  0.006890711410866631
Epoch 400 cost:  0.0055064834573056816
Epoch 500 cost:  0.004525852462882583
Epoch 600 cost:  0.003818487813949721
Epoch 700 cost:  0.003286785292047624
Epoch 800 cost:  0.002867194516425603
Epoch 900 cost:  0.002524971222504018
Epoch 1000 cost:  0.002241031241690527
Epoch 1100 cost:  0.002003883312123642
Epoch 1200 cost:  0.0018051059304428456
Epoch 1300 cost:  0.0016375135278035716
Epoch 1400 cost:  0.0014949599682621353
Epoch 1500 cost:  0.001372515805919877
Epoch 1600 cost:  0.001266425318559182
Epoch 1700 cost:  0.0011738608828628302
Epoch 1800 cost:  0.0010926403469950574
Epoch 1900 cost:  0.0010210127009311787
Final cost: 0.0009581308764275869
[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
[1 1 1 ... 1 1 1]
[[   0    2]
 [   0 2778]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
      

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [83]:
def is_inside(rect1, rect2):
    if rect1[0] > rect2[0] and (rect1[0] + rect1[2]) < (rect2[0] + rect2[2]) and rect1[1] > rect2[1] and (rect1[1] + rect1[3]) < (rect2[1] + rect2[3]):
        return True
    else:
        return False

def check_contain_another_rect(rect, rects):
    for item in rects:
        if is_inside(rect, item):
            return True
    return False

image = cv2.imread("D:/Python/HeThongThoiGianThuc/IMG.JPG")
height, width = image.shape[:2]
scaledWidth = 900
scaledHeight = int((scaledWidth * height) / width)
image = cv2.resize(image, (scaledWidth, scaledHeight), fx= 0.5, fy=0.5, interpolation= cv2.INTER_AREA)
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5,5), 0)

kernel = np.ones((3,3),dtype='uint8')
ret, im_th = cv2.threshold(im_gray, 115, 255, cv2.THRESH_BINARY_INV)
canny = cv2.Canny(im_th, 70, 170)
# canny = cv2.dilate(canny, kernel, iterations = 1)
_, ctrs, _ = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
rects = [cv2.boundingRect(ctr) for ctr in ctrs]

new_rects = []
for rect in rects:
    if rect[2] > 10 and rect[3] > 20 and rect[3] < 100 and not check_contain_another_rect(rect, rects):
        new_rects.append(rect)

# model = joblib.load("D:/Python/HeThongThoiGianThuc/mldata/digital.pkl")

new_rects = np.array(new_rects)
data_input = OrderedDict()
for rect in new_rects:
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3]/2 - leng/2)
    pt2 = int(rect[0] + rect[2]/2 - leng/2)
    roi = im_th[pt1: pt1 + leng, pt2: pt2+leng]
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    number = np.array([roi]).reshape(1, (28*28))
    data_input[(pt2,pt1, pt2 + leng, pt1+leng)] = number

for point, number in data_input.items():
    print(number.reshape(-1,1).shape)
    Z1 = np.matmul(W1, number.reshape(-1,1)) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

    predictions = np.argmax(A2, axis=0)
    cv2.rectangle(im_gray, (point[0], point[1]), (point[2], point[3]), (0, 255, 0), 1)
    cv2.putText(im_gray, str(int(predictions)), (point[0], point[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

cv2.imshow("Image", im_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)
(784, 1)


In [73]:
print(X_test.shape)

(784, 10780)
